In [1]:
import csv

In [2]:
#file reading
problem_data = []
train_subs = []

with open("train/problem_data.csv") as fproblem:
    reader = csv.reader(fproblem)
    for line in reader:
        problem_data.append(line)
    problem_data = problem_data[1:]
        
with open("train/train_submissions.csv") as ftrain:
    reader = csv.reader(ftrain)
    for line in reader:
        train_subs.append(line)
    train_subs = train_subs[1:]

In [3]:
#3rd column indicates how difficult was the problem to the user
#similarly, 4th column indicates how easy was the problem to the user

for s, subs in enumerate(train_subs):
    train_subs[s].append(7-int(subs[2]))

In [4]:
#unique users extraction
unique_users = list(set(list(map(list, zip(*(train_subs))))[0]))

In [5]:
#imputation of level_type with mode value
for i in problem_data:
    if i[1] == '':
        i[1] = 'A'

In [6]:
#user diff_profile
diff_profile = [[0]*14 for _ in unique_users]
#prev_solved problems by users(so that the same one won't be recomended again)
prev_solved = [[] for _ in unique_users]

#user diff_profile calculation
for u, user in enumerate(unique_users):
    for subs in train_subs:
        if subs[0] == user:
            prev_solved[u].append(subs[1])
            for problem in problem_data:
                if problem[0] == subs[1]:
                    diff_profile[u][ord(problem[1]) - 65] += int(subs[2])
                    break
                    
#normalization
for u, user in enumerate(diff_profile):
    denom = sum(user)
    diff_profile[u] = [i/denom for i in user]

In [7]:
#user easy_profile
easy_profile = [[0]*14 for _ in unique_users]
#prev_solved problems by users(so that the same one won't be recomended again)
prev_solved = [[] for _ in unique_users]

#user easy_profile calculation
for u, user in enumerate(unique_users):
    for subs in train_subs:
        if subs[0] == user:
            prev_solved[u].append(subs[1])
            for problem in problem_data:
                if problem[0] == subs[1]:
                    easy_profile[u][ord(problem[1]) - 65] += int(subs[3])
                    break
                    
#normalization
for u, user in enumerate(easy_profile):
    denom = sum(user)
    easy_profile[u] = [i/denom for i in user]

In [9]:
user_query = 'user_' + input('user_id(number) : ')
k  = int(input('number of recommendations required : '))
flag = 1

#profile of queried user
while(flag):
    DorE = input('recommended problems to be easier(E) or difficult(D) : ')
    if DorE == 'D':
        query_profile = diff_profile[unique_users.index(user_query)]
        flag = 0
    elif DorE == 'E':
        query_profile = easy_profile[unique_users.index(user_query)]
        flag = 0

#profile sorted according to the values in list and indices(level_type_encoded) stored seperately
sorted_profile = [index for index, value in sorted(enumerate(query_profile), reverse=True, key=lambda x:x[1])]

recommendation = []

for index in sorted_profile:
    for problem in problem_data:
        if problem[1] == chr(index+65) and problem[0] not in prev_solved[unique_users.index(user_query)]:
            recommendation.append(problem[0])

#final recommendation list calculation based on value of k
recommendation = recommendation[:k]
print('\nrecommended problems : ', recommendation)

user_id(number) : 3311
number of recommendations required : 5
recommended problems to be easier(E) or difficult(D) : E

recommended problems :  ['prob_2940', 'prob_3059', 'prob_4069', 'prob_3750', 'prob_5088']
